Everything can be done with the TPOTEstimator class. All other classes (TPOTRegressor, TPOTClassifier, TPOTSymbolicClassifier, TPOTSymbolicRegression, TPOTGeneticFeatureSetSelector, etc.) are actually just different default settings for TPOTEstimator.


By Default, TPOT will generate pipelines with a default set of classifiers or regressors as roots (this depends on whether classification is set to true or false). All other nodes are selected from a default list of selectors and transformers. Note: This differs from the TPOT1 behavior where by default classifiers and regressors can appear in locations other than the root. You can modify the the search space for leaves, inner nodes, and roots (final classifiers) separately through built in options or custom configuration dictionaries.

In this tutorial we will walk through using the built in configurations, creating custom configurations, and using nested configurations.

# ConfigSpace

Hyperparameter search spaces are defined using the [ConfigSpace package found here](https://github.com/automl/ConfigSpace). More information on how to set up a hyperparameter space can be found in their [documentation here](https://automl.github.io/ConfigSpace/main/guide.html).

In [1]:
from ConfigSpace import ConfigurationSpace
from ConfigSpace import ConfigurationSpace, Integer, Float, Categorical, Normal
from sklearn.neighbors import KNeighborsClassifier

knn_configspace = ConfigurationSpace(
    space = {

        'n_neighbors': (1, 10),
        'weights': Categorical("weights", ['uniform', 'distance']),
        'p': (1, 3),
        'metric': Categorical("metric", ['euclidean', 'minkowski']),
        'n_jobs': 1,
    }
)

hyperparameters = dict(knn_configspace.sample_configuration())
print("sampled hyperparameters")
print(hyperparameters)

knn = KNeighborsClassifier(**hyperparameters)

sampled hyperparameters
{'metric': 'minkowski', 'n_jobs': 1, 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}


# TPOT Search spaces

TPOT allows you to both hyperparameter search spaces for individual methods as well as pipeline structure search spaces. For example, TPOT can create linear pipelines, trees, or graphs. 

TPOT search spaces are found in the `search_spaces` module. There are two primary kinds of search spaces, node and pipeline. Node search spaces specify the search space of a single sklearn `BaseEstimator`. Pipeline search spaces define the possible structures for a group of node search spaces. These take in node search spaces and produce a pipeline using nodes from that search space. Since sklearn Pipelines are also `BaseEstimator`, pipeline search spaces are also technically node search spaces. Meaning that pipeline search spaces can take in other pipeline search spaces in order to define more complex structures. The primary differentiating factor bewteen node and pipeline search spaces is that pipeline search spaces must take in another search space as input to feed its individual nodes. Therefore, all search spaces eventually end in a node search space at the lowest level. Note that parameters for pipeline search spaces can differ, some take in only a single search space, some take in a list, or some take in multiple defined parameters.

search spaces can be found in tpot2.search_spaces.nodes and tpot2.search_spaces.pipelines

### node search spaces
found in tpot2.search_spaces.nodes


EstimatorNode, GeneticFeatureSelector
| Name      | Info       |
| :---        |    :----:   |
| EstimatorNode | Takes in a ConfigSpace along with the class of the method. This node will optimize the hyperparameters for a single method. |
| GeneticFeatureSelectorNode | Uses evolution to optimize a set of features, exports a basic sklearn Selector that simply selects the features chosen by the node. |




### pipeline search spaces

found in tpot2.search_spaces.pipelines

WrapperPipeline -         This search space is for wrapping a sklearn estimator with a method that takes another estimator and hyperparameters as arguments.
        For example, this can be used with sklearn.ensemble.BaggingClassifier or sklearn.ensemble.AdaBoostClassifier.


| Name      | Info       |
| :---        |    :----:   |
| ChoicePipeline | Takes in a list of search spaces. Will select one node from the search space. |
| SequentialPipeline | Takes in a list of search spaces. will produce a pipeline of Sequential length. Each step in the pipeline will correspond to the the search space provided in the same index. |
| DynamicLinearPipeline | Takes in a single search space. Will produce a linear pipeline of variable length. Each step in the pipeline will be pulled from the search space provided. |
| TreePipeline |Generates a pipeline of variable length. Pipeline will have a tree structure similar to TPOT1. |
| GraphPipeline | Generates a directed acyclic graph of variable size. Search spaces for root, leaf, and inner nodes can be defined separately if desired. |
| WrapperPipeline   | This search space is for wrapping a sklearn estimator with a method that takes another estimator and hyperparameters as arguments. For example, this can be used with sklearn.ensemble.BaggingClassifier or sklearn.ensemble.AdaBoostClassifier.        |


# Estimator node example

In [2]:
import tpot2
from ConfigSpace import ConfigurationSpace
from ConfigSpace import ConfigurationSpace, Integer, Float, Categorical, Normal
from sklearn.neighbors import KNeighborsClassifier

knn_configspace = ConfigurationSpace(
    space = {

        'n_neighbors': Integer("n_neighbors", bounds=(1, 10)),
        'weights': Categorical("weights", ['uniform', 'distance']),
        'p': Integer("p", bounds=(1, 3)),
        'metric': Categorical("metric", ['euclidean', 'minkowski']),
        'n_jobs': 1,
    }
)


knn_node = tpot2.search_spaces.nodes.EstimatorNode(
    method = KNeighborsClassifier,
    space = knn_configspace,
)

/Users/gabrielketron/tpot2_addimputers/env2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You can sample generate an individual with the generate() function. This individual samples from the search space as well as provides mutation and crossover functions to modify the current sample.

Note that ConfigurationSpace does not support None as a parameter. Instead, use the special string "\<NONE\>". TPOT will automatically replace instances of this string with the Python None.

In [3]:
knn_individual = knn_node.generate()

print("sampled hyperparameters")
print(knn_individual.hyperparameters)
knn_individual.mutate() # mutate the individual
print("mutated hyperparameters")
print(knn_individual.hyperparameters)

sampled hyperparameters
{'metric': 'minkowski', 'n_jobs': 1, 'n_neighbors': 4, 'p': 1, 'weights': 'distance'}
mutated hyperparameters
{'metric': 'euclidean', 'n_jobs': 1, 'n_neighbors': 10, 'p': 1, 'weights': 'uniform'}


In TPOT2, crossover only modifies the individual calling the crossover function, the second individual remains the same

In [4]:
knn_individual1 = knn_node.generate()
knn_individual2 = knn_node.generate()

print("original hyperparameters for individual 1")
print(knn_individual1.hyperparameters)

print("original hyperparameters for individual 2")
print(knn_individual2.hyperparameters)

print()

knn_individual1.crossover(knn_individual2) # crossover the individuals
print("post crossover hyperparameters for individual 1")
print(knn_individual1.hyperparameters)
print("post crossover hyperparameters for individual 2")
print(knn_individual2.hyperparameters)



original hyperparameters for individual 1
{'metric': 'minkowski', 'n_jobs': 1, 'n_neighbors': 8, 'p': 3, 'weights': 'uniform'}
original hyperparameters for individual 2
{'metric': 'euclidean', 'n_jobs': 1, 'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}

post crossover hyperparameters for individual 1
{'metric': 'euclidean', 'n_jobs': 1, 'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}
post crossover hyperparameters for individual 2
{'metric': 'euclidean', 'n_jobs': 1, 'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}


All search spaces have an export_pipeline function that returns an sklearn `BaseEstimator`

In [5]:
knn_individual1.export_pipeline()

KNeighborsClassifier(metric='euclidean', n_jobs=1, n_neighbors=9, p=1)

If a dictionary of parameters is passed instead of of a ConfigSpace, then the hyperparameters will be fixed and not learned.

In [6]:
import tpot2
from ConfigSpace import ConfigurationSpace
from ConfigSpace import ConfigurationSpace, Integer, Float, Categorical, Normal
from sklearn.neighbors import KNeighborsClassifier

space = {

    'n_neighbors':10,
}

knn_node = tpot2.search_spaces.nodes.EstimatorNode(
    method = KNeighborsClassifier,
    space = space,
)

knn_node.generate().export_pipeline()

KNeighborsClassifier(n_neighbors=10)

# Pipeline Search Spaces

## choice search space

The simplest pipeline search space is the ChoicePipeline. This takes in a list of search spaces and simply selects and samples from one. In this example, we will construct a search space that takes in several options for a classifier.

In [7]:
import tpot2
from ConfigSpace import ConfigurationSpace
from ConfigSpace import ConfigurationSpace, Integer, Float, Categorical, Normal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

knn_configspace = ConfigurationSpace(
    space = {

        'n_neighbors': Integer("n_neighbors", bounds=(1, 10)),
        'weights': Categorical("weights", ['uniform', 'distance']),
        'p': Integer("p", bounds=(1, 3)),
        'metric': Categorical("metric", ['euclidean', 'minkowski']),
        'n_jobs': 1,
    }
)

lr_configspace = ConfigurationSpace(
        space = {
            'solver': Categorical("solver", ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
            'penalty': Categorical("penalty", ['l1', 'l2']),
            'dual': Categorical("dual", [True, False]),
            'C': Float("C", bounds=(1e-4, 1e4), log=True),
            'class_weight': Categorical("class_weight", ['balanced']),
            'n_jobs': 1,
            'max_iter': 1000,
        }
    )

dt_configspace = ConfigurationSpace(
        space = {
            'criterion': Categorical("criterion", ['gini', 'entropy']),
            'max_depth': Integer("max_depth", bounds=(1, 11)),
            'min_samples_split': Integer("min_samples_split", bounds=(2, 21)),
            'min_samples_leaf': Integer("min_samples_leaf", bounds=(1, 21)),
            'max_features': Categorical("max_features", ['sqrt', 'log2']),
            'min_weight_fraction_leaf': 0.0,
        }
    )

knn_node = tpot2.search_spaces.nodes.EstimatorNode(
    method = KNeighborsClassifier,
    space = knn_configspace,
)

lr_node = tpot2.search_spaces.nodes.EstimatorNode(
    method = LogisticRegression,
    space = lr_configspace,
)

dt_node = tpot2.search_spaces.nodes.EstimatorNode(
    method = DecisionTreeClassifier,
    space = dt_configspace,
)

classifier_node = tpot2.search_spaces.pipelines.ChoicePipeline(
    search_spaces=[
        knn_node,
        lr_node,
        dt_node,
    ]
)


tpot2.search_spaces.pipelines.ChoicePipeline(
    search_spaces = [
        tpot2.search_spaces.nodes.EstimatorNode(
            method = KNeighborsClassifier,
            space = knn_configspace,
            ),
        tpot2.search_spaces.nodes.EstimatorNode(
            method = LogisticRegression,
            space = lr_configspace,
        ),
        tpot2.search_spaces.nodes.EstimatorNode(
            method = DecisionTreeClassifier,
            space = dt_configspace,
        ),
    ]
)

Search space objects provided by pipeline search spaces work the same as with node search spaces. Note that crossover only works when both individuals have sampled the same method. 

In [8]:
classifier_individual = classifier_node.generate()

print("sampled pipeline")
classifier_individual.export_pipeline()

sampled pipeline


LogisticRegression(C=0.0055620037529, class_weight='balanced', dual=True,
                   max_iter=1000, n_jobs=1)

In [9]:
print("mutated pipeline")
classifier_individual.mutate()
classifier_individual.export_pipeline()

mutated pipeline


LogisticRegression(C=0.0025825951933, class_weight='balanced', max_iter=1000,
                   n_jobs=1)

TPOT2 also comes with predefined search spaces. The current search spaces were adapted from a combination of the original TPOT package as well as the search spaces used in [AutoSklearn](https://github.com/automl/auto-sklearn/tree/development/autosklearn/pipeline/components). The helper function `tpot2.config.get_search_space` takes in a string or a list of strings, and returns either a EstimatorNode or a ChoicePipeline,respectively. 

strings can correspond to individual methods. Tehre are also special strings that return predefined lists of methods. 

Special strings are "selectors", "classifiers", "transformers"

EstimatorNode, GeneticFeatureSelector
| Special String     | Included methods      |
| :---        |    :----:   |
| "selectors" | "SelectFwe", "SelectPercentile", "VarianceThreshold", "RFE", "SelectFromModel" |
| "classifiers" | "LogisticRegression", "KNeighborsClassifier", "DecisionTreeClassifier", "SVC", "LinearSVC", "RandomForestClassifier", "GradientBoostingClassifier", "XGBClassifier", "LGBMClassifier", "ExtraTreesClassifier", "SGDClassifier", "MLPClassifier", "BernoulliNB", "MultinomialNB"  |
| "transformers" | "Binarizer", "Normalizer", "PCA", "ZeroCount", "OneHotEncoder", "FastICA", "FeatureAgglomeration", "Nystroem", "RBFSampler" |

In [10]:
#same pipeline search space as before.
classifier_choice = tpot2.config.get_search_space(["KNeighborsClassifier", "LogisticRegression", "DecisionTreeClassifier"])

print("sampled pipeline 1")
classifier_choice.generate().export_pipeline()

sampled pipeline 1


DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=18, min_samples_leaf=20, min_samples_split=19)

In [11]:
print("sampled pipeline 2")
classifier_choice.generate().export_pipeline()

sampled pipeline 2


DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=13, max_features='log2', min_samples_leaf=9,
                       min_samples_split=1)

In [12]:
#search space for all classifiers
classifier_choice = tpot2.config.get_search_space("classifiers")

print("sampled pipeline 1")
classifier_choice.generate().export_pipeline()

sampled pipeline 1


MLPClassifier(alpha=0.0002659149506, hidden_layer_sizes=[413, 413],
              learning_rate_init=0.0037838163328, n_iter_no_change=32)

In [13]:
print("sampled pipeline 2")
classifier_choice.generate().export_pipeline()

sampled pipeline 2


SGDClassifier(alpha=0.0022145218809, class_weight='balanced',
              eta0=0.9417181204716, l1_ratio=0.7346312639,
              learning_rate='invscaling', loss='modified_huber', n_jobs=1,
              penalty='elasticnet', power_t=0.1918481668943)

# Sequential Example

SequentialPipelines are of fixed length and sample from a predefined distribution for each step. Here is an example of the form Selector-Transformer-Classifer

In [14]:
stc_pipeline = tpot2.search_spaces.pipelines.SequentialPipeline([
    tpot2.config.get_search_space("selectors"), 
    tpot2.config.get_search_space("transformers"),
    tpot2.config.get_search_space("classifiers"),
    
])

print("sampled pipeline")
print(dir(stc_pipeline.search_spaces[0]))


sampled pipeline
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'generate', 'search_spaces']


In [15]:
print("sampled pipeline")
stc_pipeline.generate().export_pipeline()

sampled pipeline


Pipeline(steps=[('variancethreshold',
                 VarianceThreshold(threshold=0.1014392124361)),
                ('powertransformer', PowerTransformer()),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(early_stopping=True,
                                                l2_regularization=1.2249e-09,
                                                learning_rate=0.6489832826848,
                                                max_features=0.8976523035715,
                                                max_leaf_nodes=1237,
                                                min_samples_leaf=156,
                                                n_iter_no_change=12, tol=0.0001,
                                                validation_fraction=0.2704236550776))])

# Optimize Search Space with TPOTEstimator

Once you have constructed a search space, you can use TPOTEstimator to optimize a pipeline within that space.

In [16]:
import tpot2
import numpy as np
import sklearn
import sklearn.datasets

# create dummy dataset
X, y = sklearn.datasets.make_classification(n_samples=200, n_features=10, n_classes=2)

# train test split
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.5)



graph_search_space = tpot2.search_spaces.pipelines.SequentialPipeline([
    tpot2.config.get_search_space("imputers"),
    tpot2.config.get_search_space("classifiers"),
])


est = tpot2.TPOTEstimator(
    scorers = ["roc_auc"],
    scorers_weights = [1],
    classification = True,
    cv = 5,
    search_space = graph_search_space,
    population_size= 10,
    generations = 5,
    max_eval_time_seconds = 60*5,
    verbose = 2,
)

est.fit(X_train, y_train)





Generation: 100%|██████████| 5/5 [00:20<00:00,  4.08s/it]


TPOTEstimator(classification=True, generations=5, max_eval_time_seconds=300,
              population_size=10, scorers=['roc_auc'], scorers_weights=[1],
              search_space=<tpot2.search_spaces.pipelines.sequential.SequentialPipeline object at 0x315b2a530>,
              verbose=2)

In [23]:
est.fitted_pipeline_[0].transform(X_train)

array([[ 6.40848553e-01,  1.07478030e+00,  8.48278289e-01,
         1.64174992e+00,  1.04449833e+00,  2.82291611e-03,
         1.21694528e+00,  1.00522500e+00,  1.71284339e-01,
         1.74276261e+00],
       [ 6.76851770e-01,  1.98562268e+00, -1.36241965e+00,
        -1.53279899e+00, -8.76405588e-01,  1.72966897e-01,
         1.37728789e+00,  2.16227089e+00, -1.29796743e+00,
        -1.06460957e+00],
       [-2.49588215e-01, -5.09598733e-01, -2.49622957e-01,
        -6.75934610e-01, -3.17362866e-01, -3.18312723e-02,
        -7.76017786e-01, -5.26739928e-01,  5.60692885e-01,
        -8.15763723e-01],
       [-1.61872978e+00, -4.95058413e-01,  1.79342032e+00,
         1.49116010e+00, -4.85825921e-01, -3.12340472e-01,
         1.10633655e-01, -1.26967515e+00,  9.18235941e-01,
         5.74189680e-01],
       [ 2.42077444e+00, -1.01661461e-01, -6.75292494e-01,
        -8.20635537e-01, -1.64569837e+00,  7.59419282e-02,
         9.20017384e-01,  7.90102836e-01,  5.72260764e-01,
        -6.

In [24]:
# score the model

auroc_scorer = sklearn.metrics.get_scorer("roc_auc")
auroc_score = auroc_scorer(est, X_test, y_test)

print("auroc score", auroc_score)

auroc score 0.9127272727272727


In [27]:
#plot the best pipeline
est.fitted_pipeline_[0]

KNNImputer(n_neighbors=39)

In [ ]:
est

TPOTEstimator(classification=True, generations=5, max_eval_time_seconds=300,
              population_size=10, scorers=['roc_auc'], scorers_weights=[1],
              search_space=<tpot2.search_spaces.pipelines.graph.GraphPipeline object at 0x3148a7280>,
              verbose=2)

# Combined Search Space Example

In [ ]:
from tpot2.search_spaces.pipelines import *
from tpot2.config import get_search_space

selectors = get_search_space(["selectors","selectors_classification", "Passthrough"])
estimators = get_search_space(["classifiers"])


# this allows us to wrap the classifiers in the EstimatorTransformer
# this is necessary so that classifiers can be used inside of sklearn pipelines
wrapped_estimators = WrapperPipeline(tpot2.builtin_modules.EstimatorTransformer, {}, estimators)

scalers = get_search_space(["scalers","Passthrough"])

transformers_layer =UnionPipeline([
                        ChoicePipeline([
                            DynamicUnionPipeline(get_search_space(["transformers"])),
                            get_search_space("SkipTransformer"),
                        ]),
                        get_search_space("Passthrough")
                        ]
                    )

inner_estimators_layer = UnionPipeline([
                            ChoicePipeline([
                                DynamicUnionPipeline(wrapped_estimators),
                                get_search_space("SkipTransformer"),
                            ]),
                            get_search_space("Passthrough")]
                        )


search_space = SequentialPipeline(search_spaces=[
                                        scalers,
                                        selectors, 
                                        transformers_layer,
                                        inner_estimators_layer,
                                        estimators,
                                        ])

est = tpot2.TPOTEstimator(
    scorers = ["roc_auc"],
    scorers_weights = [1],
    classification = True,
    cv = 5,
    search_space = search_space,
    population_size= 10,
    generations = 5,
    max_eval_time_seconds = 60*5,
    verbose = 2,
)

est.fit(X_train, y_train)

Generation: 100%|██████████| 5/5 [00:22<00:00,  4.60s/it]
/Users/gabrielketron/tpot2_addimputers/env2/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:595: UserWarning: n_components is too large: it will be set to 3
  warnings.warn(


TPOTEstimator(classification=True, generations=5, max_eval_time_seconds=300,
              population_size=10, scorers=['roc_auc'], scorers_weights=[1],
              search_space=<tpot2.search_spaces.pipelines.sequential.SequentialPipeline object at 0x328bb4b50>,
              verbose=2)

In [ ]:
est.fitted_pipeline_

Pipeline(steps=[('robustscaler',
                 RobustScaler(quantile_range=(0.0991522796349, 0.86486907733))),
                ('selectfwe', SelectFwe(alpha=0.0130739725344)),
                ('featureunion-1',
                 FeatureUnion(transformer_list=[('featureunion',
                                                 FeatureUnion(transformer_list=[('fastica',
                                                                                 FastICA(algorithm='deflation',
                                                                                         n_components=24)),
                                                                                ('binarizer',
                                                                                 Binarizer(threshold=0.2602740471335))])),
                                                ('passthrough',
                                                 Passthrough())])),
                ('featureunion-2',
                 FeatureUnion(transformer_list=[('skiptransformer',
                                                 SkipTransformer()),
                                                ('passthrough',
                                                 Passthrough())])),
                ('lgbmclassifier',
                 LGBMClassifier(max_depth=7, n_estimators=74, n_jobs=1,
                                num_leaves=225, verbose=-1))])